## Desarrollo de Inteligencia Artifical Generativa con LLM
---
Dado que el equipo disponible no cuenta con los requerimientos mínimos para ejecutar el modelo deseado de forma local, se recurre a Colab para mirar la implementación del código que extrae la información del texto extraído de documentos `.docx` o `.pdf`.


In [ ]:
# fichero modelo.py

from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers.generation.utils import GenerateDecoderOnlyOutput, GenerateBeamDecoderOnlyOutput
import torch


def modelo_LLM_inicializacion(device: str = "cpu"):

    """
    modelo_LLM_inicializacion inicializa el modelo Qwen 1.5.
    parámetro 1: cpu o cuda dependiendo del dispositivo de ejecución
    devuelve: modelo y tokenizador
    """


    SMALL_CHAT_MODEL_CON_CASTELLANO =  "Qwen/Qwen1.5-1.8B-Chat"
    DEVICE = device # DEVICE = "cuda:0" para ejecución con GPU, DEVICE = "cpu" para ejecución en cpu, el cero de cuda:0 es para especficar la tarjeta gráfica a utilizar, el 0 inica que es la principal

    model = AutoModelForCausalLM.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO).to(DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO)

    return model, tokenizer


def crear_prompt_chat_model(
        mensaje_usuario: str,
        tokenizer: AutoTokenizer,
        mensaje_sistema: str = "You are an unhelpful assistant."
) -> str:

    """
    crear_prompt_chat_model crea la estructura de prompt específica del prompt para que el modelo Qwen 1.5 ejecute las instrucciones deseadas
    parámetro 1: Mensaje de usuario que es el texto sobre el cual se ejecutarán las instrucciones.
    parámetro 2: Tokenizador seleccionado.
    parámetro 3: Instrucciones para el modelo.
    devuelve: Estructura del prompt necesaria para que el modelo se ejecute correctamente.
    """


    messages = [{"role": "system", "content": mensaje_sistema}]

    messages.append({"role": "user", "content": mensaje_usuario})
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    return prompt


def responde_al_mensaje(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        prompt: str,
        DEVICE: str,
        streamer: TextStreamer = None,
        gen_kwargs: dict = None,
        return_scores: bool = False,
    ) -> list[str] | tuple[list[str], GenerateDecoderOnlyOutput]:

    """
    responde_al_mensaje realiza la ejecución del modelo con los parámetros especificados.
    parámetro 1: Modelo.
    parámetro 2: Tokenizer,
    parámetro 3: Prompt,
    parámetro 4: Dispocitivo (DEVICE),
    parámetro 5: Streamer,
    parámetro 6: Parámetros del modelo,
    parámetro 7: Devolver puntuaciones,
    devuelve: Salida del modelo.
    """

    gen_kwargs = gen_kwargs or {}
    if return_scores:
        gen_kwargs["return_dict_in_generate"] = True
        gen_kwargs["output_scores"] = True

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    model_output = model.generate(
        input_ids,
        **gen_kwargs,
        streamer=streamer,
    )
    if isinstance(model_output, GenerateDecoderOnlyOutput):
        response_tokens = model_output.sequences
    elif isinstance(model_output, torch.Tensor):
        response_tokens = model_output
    elif isinstance(model_output, GenerateBeamDecoderOnlyOutput):
        response_tokens = model_output.sequences
    else:
        raise ValueError(f"El modelo ha devuelto un tipo inesperado: {type(model_output)}")


    responses_txt = tokenizer.batch_decode(response_tokens[:,len(input_ids[0]):], skip_special_tokens=True)

    return responses_txt, model_output


def streamer_gen_kwargs(tokenizer, temperatura, maximo_tokens):
    """
    streamer_gen_kwargs crea los parámetros adiconales para el modelo.
    parámetro 1: Tokenizador,
    parámetro 2: Temperatura,
    parámetro 3: Máximo de Tokens de salida
    devuelve: streamer y gen_kwargs
    """

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    gen_kwargs = {"temperature": temperatura, "top_k": 1, "max_new_tokens": maximo_tokens}

    return streamer, gen_kwargs

In [ ]:
# prompts

from datetime import date

hoy = date.today()


def instruccion_1():

    instrucciones_1 = '''
    Eres una inteligencia artificial especializada en revisar hojas de vida de candidatos para una vacante en una empresa.

    Recibes una hoja de vida y tu tarea es extraer el nombre del candidato, el teléfono del candidato y el correo electrónico del candidato con el siguiente formato JSON:

    {{
      "Nombre": <Nombre del candidato>,
      "Telefono": <Los 10 dígitos del teléfono del candidato sin espacios>,
      "Correo": <Correo Electrócino del candidato>
    }}
    '''
    return instrucciones_1


def instruccion_2():

    instruccion_2 = '''
    Identify all the dates associated to job positions in the CV, your answer should be as short as possible.
    '''

    return instruccion_2


def instruccion_3():

    instruccion_3 = f'''
    Identifica la cantidad de parejas de años presentes en el texto recibido. Extrae únicamente los años encontrados en el texto con el siguiente formato:

    {{
      "Años": <Parejas de años encontradas>,
      "Texto": <Textos encontrados sin los años>.
    }}
    '''

    return instruccion_3

In [ ]:
texto_CV = '''
Sergio Alejandro Alvarado Parada
Celular: +57 3017394784
Correo electrónico: sergio.ing92@gmail.com
LinkedIn: Sergio Alejandro Alvarado Parada
DataScientist | DataAnalyst | MachineLearning | Python | SQL | R
Perfil Profesional
Ingeniero bilingüe y Magíster en Big Data y Data Science, con experiencia en el área de análisis de
datos, aplicando técnicas de minería de datos, análisis estadístico, implementación de modelos de
machine learning, visualización de datos. Habilidades en la manipulación y limpieza de datos,
creación de algoritmos personalizados, desarrollo de modelos predictivos y de segmentación, así
como en la identificación de patrones y tendencias a través de técnicas estadísticas. Capacidad
para aplicar métodos analíticos y herramientas de programación con el fin de extraer información y
generar conocimientos accionables para la toma de decisiones estratégicas. Con dominio en el uso
de herramientas como Python, R, SQL, Pyspark y MongoDB para el análisis de datos y la generación
de dashboard. Con interés en el área de data science para impulsar la toma de decisiones
informadas y mejorar la eficiencia operativa y el rendimiento empresarial.
Habilidades
● Estadística.
● Programación.
● Python para ciencia de datos (Pandas, Numpy, Matplotlib, scikit-learn, Beautiful Soup)
● R (ggplot, dplyr, earth, spdep).
● SQL.
● Pyspark.
● MongoDB.
Experiencia
Data Scientist & Data Analyst | Freelance Febrero 2022 – Actualidad
● Realizar la extracción, transformación y load (ETL) de información de un conjunto de datos para
su análisis, visualización e implementación en modelos de machine learning.
● Diseñar árboles de decisión, modelos de regresión logística y otros algoritmos para gestionar
procesos de regresión y clasificación por medio de algoritmos de machine learning enfocados
en entrenamiento supervisado.
● Realizar la migración de bases de datos relacionales de un servidor local a un servidor en la
nube por medio de Amazon Web Services (AWS) para mejorar la eficiencia en el
almacenamiento de datos.
● Extraer información de páginas web mediante Web Scraping para automatizar la recolección
de datos agilizando el proceso de obtención de información relevante.
Logros
● Conseguí optimizar en un 99.3% el tiempo necesario para la complementación de información
de 1.350 archivos por medio de la creación de un algoritmo en Python que permitió
automatizar el proceso.
● Logré automatizar 100% el proceso de recolección de información a través de la creación de
un script en Python para extraer información de páginas web que no pueden descargarse
directamente como ficheros por medio de Web Scraping.
● Identifiqué 3 grupos de consumidores con patrones de compra similares analizando el
comportamiento y características de usuarios de una tienda retail a partir del algoritmo de
Machine Learning KNN (K-Nearest Neighbor).
Account Change Specialist| Amazon | Junio 2022 – Actualidad
● Realizar procesos de validación de datos para identificar fraudes y asegurar la integridad y
seguridad de las cuentas de Amazon de clientes en Estados Unidos y Canadá.
● Asegurar la confidencialidad de la información de los usuarios.
Logros
● Garanticé la integridad y seguridad del 99,9% de las cuentas bajo mi responsabilidad evitando
el robo de información personal y bancaria.
● Evité el uso indebido por parte de estafadores por sumas superiores a USD$50,000.
Ingeniero de Diseño | Ingetec S.A. Febrero 2018 – Diciembre 2021
● Analizar datos a través de herramientas estadísticas para la identificación de la viabilidad de
proyectos.
● Crear algoritmos de optimización de procesos con el uso de lenguajes de programación R y
Python para la reducción de tiempos de procesamiento y eliminación de errores humanos.
● Crear visualizaciones de la información analizada con R, Python y Excel para resumir y permitir
entender más fácilmente la cantidad de datos analizados.
● Programación de test estadísticos en Excel, R y Python para pruebas paramétricas y no
paramétricas con el fin de identificar la distribución de los datos.
Logros
● Aumenté la productividad de la empresa en los análisis de datos en un 100% gracias a la
implementación de programas de automatización y creación de Dashboards a partir de
Python y librerías como pandas y xlsxwriter.
● Reduje a 0 la cantidad de errores presentados en etapas de manipulación y transformación de
datos implementando procesos de automatización y visualización en Python.
● Reduje el tiempo necesario en la organización y manipulación de datos en un 95%, a través de
la programación de algoritmos para manipular y procesar datos en etapas tempranas del
proyecto.
Educación
Máster en Big Data y Data Science Noviembre 2022 – Noviembre 2023
Universitat de Barcelona
Ingeniería Civil Enero 2010 – Septiembre 2015
Escuela Colombiana de Ingeniería Julio Garavito
Educación complementaria
Diplomado Inteligencia Artificial para las Ciencias y las Ingenierías Enero 2022 – Mayo 2022
Universidad de Antioquia
Diplomado Machine Learning and Data Science Febrero 2021 – Junio 2021
Universidad Nacional de Colombia Sede Bogotá
Curso Big Data Mayo 2021 – Junio 2021
Universidad Nacional de Colombia Sede Bogotá
Curso Introducción al Aprendizaje Computacional con Python Abril 2021 – Mayo 2021
Universidad Nacional de Colombia Sede Bogotá
Curso Análisis y Visualización de Datos con Python Febrero 2021 – Marzo 2021
Universidad Nacional de Colombia Sede Bogotá
Curso de Business Intelligence con Python Septiembre 2019 – Octubre 2019
Pontificia Universidad Javeriana
Idiomas
Inglés B2
'''

In [ ]:
# Fichero main

model, tokenizer = modelo_LLM_inicializacion()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Fichero posprocesado.py

import json
import pandas as pd


def posprocesado(respuesta):

    """
    posprocesado convierte la salida del modelo (str) en un diccionario o lista según sea el caso.
    parámetro 1: Salida en formato str del modelo.
    devuelve: Diccionario o Lista de Python
    """

    return json.loads(respuesta)


def exportar_excel(informacion_candidato, experiencia_profesional):

    """
    exportar_excel crea un archivo de excel con la información organizada extraida por el modelo.
    parámetro 1: Diccionario con la información básica de contacto del candidato.
    parámetro 2: Diccionario o lista de diccionarios con la experiencia profesional del candidado.
    devuelve: Una confirmación de la creación del archivo de excel o una mensaje de error.
    """

    experiencia = experiencia_profesional["Años"]
    informacion = informacion_candidato

    for diccionario in experiencia:
        anios = abs(int(diccionario["Años"].split("-")[0]) - int(diccionario["Años"].split("-")[1])) + 1
        diccionario["Años"] = anios

    columnas = ["Nombre", "Teléfono", "Correo", "Lista Experiencia", "Año Mayor Experiencia", "Puesto Mayor Experiencia"]
    datos = list(informacion.values())
    datos.append(experiencia)

    anios_mayor_exp = 0
    puesto_mayor_exp = ""

    for diccionario in experiencia:
        if diccionario["Años"] >= anios_mayor_exp:
            anios_mayor_exp = diccionario["Años"]
            puesto_mayor_exp = diccionario["Texto"]

    datos.append(anios_mayor_exp)
    datos.append(puesto_mayor_exp)

    df = pd.DataFrame(columns=columnas)
    df.loc[len(df)] = datos

    return df

In [ ]:
tarea_1 = instruccion_1()

prompt1 = crear_prompt_chat_model(texto_CV, tokenizer, tarea_1)
streamer1, gen_kwargs1 = streamer_gen_kwargs(tokenizer, 0.01, 55)

response1, model_output1 = responde_al_mensaje(model, tokenizer, prompt1, DEVICE="cpu", streamer=streamer1, gen_kwargs=gen_kwargs1)

{
  "Nombre": "Sergio Alejandro Alvarado Parada",
  "Telefono": "+57 3017394784",
  "Correo electrónico": "sergio.ing92@gmail.com"
}


In [ ]:
tarea_2 = instruccion_2()

prompt2 = crear_prompt_chat_model(texto_CV, tokenizer, tarea_2)
streamer2, gen_kwargs2 = streamer_gen_kwargs(tokenizer, 0.01, 100)

response2, model_output2 = responde_al_mensaje(model, tokenizer, prompt2, DEVICE="cpu", streamer=streamer2, gen_kwargs=gen_kwargs2)

2022 - Present
Big Data and Data Science Master's from Universitat de Barcelona
2010 - 2015
Colombian Engineer
2022 - Present
Machine Learning and Data Science Diploma from Universidad Nacional de Colombia
2021 - 2021
Machine Learning and Data Science Degree from Universidad Nacional de Colombia
2021 - 2021
Introduction to Computer Science with Python Course from Pontificia


In [ ]:
experiencia = response2[0].replace("Present", str(date.today().year))
print(experiencia)

2022 - 2024
Big Data and Data Science Master's from Universitat de Barcelona
2010 - 2015
Colombian Engineer
2022 - 2024
Machine Learning and Data Science Diploma from Universidad Nacional de Colombia
2021 - 2021
Machine Learning and Data Science Degree from Universidad Nacional de Colombia
2021 - 2021
Introduction to Computer Science with Python Course from Pontificia


In [ ]:
tarea_3 = instruccion_3()

prompt3 = crear_prompt_chat_model(experiencia, tokenizer, tarea_3)
streamer3, gen_kwargs3 = streamer_gen_kwargs(tokenizer, 0.01, 500)

response3, model_output3 = responde_al_mensaje(model, tokenizer, prompt3, DEVICE="cpu", streamer=streamer3, gen_kwargs=gen_kwargs3)

{
  "Años": [
    {
      "Años": "2022 - 2024",
      "Texto": "Big Data and Data Science Master's from Universitat de Barcelona"
    },
    {
      "Años": "2010 - 2015",
      "Texto": "Colombian Engineer"
    },
    {
      "Años": "2022 - 2024",
      "Texto": "Machine Learning and Data Science Diploma from Universidad Nacional de Colombia"
    },
    {
      "Años": "2021 - 2021",
      "Texto": "Machine Learning and Data Science Degree from Universidad Nacional de Colombia"
    }
  ],
  "Texto": ""
}


In [ ]:
print(response3[0])

{
  "Años": [
    {
      "Años": "2022 - 2024",
      "Texto": "Big Data and Data Science Master's from Universitat de Barcelona"
    },
    {
      "Años": "2010 - 2015",
      "Texto": "Colombian Engineer"
    },
    {
      "Años": "2022 - 2024",
      "Texto": "Machine Learning and Data Science Diploma from Universidad Nacional de Colombia"
    },
    {
      "Años": "2021 - 2021",
      "Texto": "Machine Learning and Data Science Degree from Universidad Nacional de Colombia"
    }
  ],
  "Texto": ""
}


In [ ]:
info_candidato = posprocesado(response1[0])
info_candidato

{'Nombre': 'Sergio Alejandro Alvarado Parada',
 'Telefono': '+57 3017394784',
 'Correo electrónico': 'sergio.ing92@gmail.com'}

In [ ]:
experiencia_profesional = posprocesado(response3[0])
experiencia_profesional

{'Años': [{'Años': '2022 - 2024',
   'Texto': "Big Data and Data Science Master's from Universitat de Barcelona"},
  {'Años': '2010 - 2015', 'Texto': 'Colombian Engineer'},
  {'Años': '2022 - 2024',
   'Texto': 'Machine Learning and Data Science Diploma from Universidad Nacional de Colombia'},
  {'Años': '2021 - 2021',
   'Texto': 'Machine Learning and Data Science Degree from Universidad Nacional de Colombia'}],
 'Texto': ''}

In [ ]:
dataframe = exportar_excel(info_candidato, experiencia_profesional)

In [ ]:
dataframe

,Nombre,Teléfono,Correo,Lista Experiencia,Año Mayor Experiencia,Puesto Mayor Experiencia
0,Sergio Alejandro Alvarado Parada,+57 3017394784,sergio.ing92@gmail.com,"[{'Años': 3, 'Texto': 'Big Data and Data Scien...",6,Colombian Engineer


---